In [0]:
import os
import string
import random
import numpy as np
import pickle
import tensorflow_hub as hub
from wordsegment import load, segment
from collections import Counter
from scipy import spatial
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [0]:
# preprocessing tag content
# separate words of each tag, remove punctuation, etc
# make "#catoftheday" -> "cat of the day"
def collect_tags_and_decomposition(path):
  tags = pickle.load(open(path, "rb"))
  load()
  for key, value in tags.items():  
    cur_list = []
    for v in value[0]:
      cur = v.split('#')[1].lower()
      cur = segment(cur)
      cur = ' '.join(cur)
      if cur:
        cur_list.append(cur)
    tags[key] = cur_list
  return tags
tags = collect_tags_and_decomposition("final-imgs.pickle")
print("How many tags in total: ")
print(len(tags))

In [0]:
# group all tags and select popular tags
# select images with at least one of the popular tags
# save {image id : list of popular tags} in dictionary
def filter_images_with_popular_tags(tags):
  popular_tags = [v for value in tags.values() for v in value]
  popular_tags = Counter(popular_tags).most_common(500)
  popular_tags = [k[0] for k in popular_tags]
  image_to_tag_mapping = dict()
  for key, value in tags.items():
    target_value = []
    for v in value:
      if v in popular_tags:
        target_value.append(v)
    if target_value:
      image_to_tag_mapping[key] = target_value
  return popular_tags, image_to_tag_mapping

popular_tags, image_to_tag_mapping = filter_images_with_popular_tags(tags)
print("How many popular tags as candidates:")
print(len(popular_tags))
print("How many post (image & tag pairs) after filtering out unpopular tags: ")
print(len(image_to_tag_mapping.keys()))

In [0]:
# encode each popular tag into a 512 vector using universal sentence encoding 
# store the encoding in a list with the same length as popular_tags
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def sentence_encode_tags(popular_tags):
  return embed(popular_tags)
encode_tags = sentence_encode_tags(popular_tags)
print("Dimension of text embedding:")
print(len(encode_tags))
tree = spatial.KDTree(encode_tags)   # important !!

In [0]:
# make one predictions of each selected image (with at least one popular tag) using pre-trained VGG16 model
def make_vgg_predictions(directory, image_to_tag_mapping_keys):
	vgg_model = VGG16()
	vgg_model = Model(inputs = vgg_model.inputs, outputs = vgg_model.layers[-1].output)
	vgg_predictions = dict()
	temp = 0
	for name in os.listdir(directory):
		if temp % 100 == 0:
			print(temp)
		temp += 1 
		filename = directory + name
		name = name.split('.')[0]
		if name not in image_to_tag_mapping_keys:
			continue
		image = load_img(filename, target_size=(224, 224))
		image = img_to_array(image)  # (224, 224, 3)
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		image = preprocess_input(image)
		feature = vgg_model.predict(image, verbose=0)
		label = decode_predictions(feature)[0]
		vgg_predictions[name] = label[0][1].replace("_", " ")
	return vgg_predictions
vgg_predictions = make_vgg_predictions('images/', image_to_tag_mapping.keys())
print("How many images are classified using CNN model:")
print(len(vgg_predictions))

In [0]:
# encode prediction of each image into a 512 vector using universal sentence encoding 
def sentence_encode_vgg_predictions(vgg_predictions):
  encode_predictions = dict()
  for key, value in vgg_predictions.items():
    encode_predictions[key] = embed([value])[0]
  return encode_predictions
encode_predictions = sentence_encode_vgg_predictions(vgg_predictions)
print("How many image classifications are encoded as vector:")
print(len(encode_predictions))
print("Check the vector dimension:")
print(len(list(encode_predictions.values())[0]))

In [0]:
# find the top k nearest encoded popular tag for each encoded prediction
def euclidean_find_nearest_popular_tags(popular_tags, tree, encode_predictions):
  euclidean_final_tag_predictions = dict()
  for key, value in encode_predictions.items():
    euclidean_final_tag_predictions[key] = [popular_tags[i] for i in tree.query(value, k = 36)[1]]
  return euclidean_final_tag_predictions
euclidean_final_tag_predictions = euclidean_find_nearest_popular_tags(popular_tags, tree, encode_predictions)
print("How many images are predicted with a list of tags:")
print(len(euclidean_final_tag_predictions))

In [0]:
# if the top k prediction has at least one match with the true tag set, count as correct
# calculate the prediction accuracy
# used for both euclidean and logistic regression  !!!
def calculate_correctness(image_to_tag_mapping, final_tag_predictions):
  correct_num = 0
  for key, value in final_tag_predictions.items():
    if not set(value).isdisjoint(image_to_tag_mapping[key]):
      correct_num += 1
  return correct_num / len(final_tag_predictions)
euclidean_correctness = calculate_correctness(image_to_tag_mapping, euclidean_final_tag_predictions)
print("The model prediction accuracy is:")
print(euclidean_correctness)

In [0]:
def euclidean_predict_one_image(filename, tree, popular_tags):
  vgg_model = VGG16()
  vgg_model = Model(inputs = vgg_model.inputs, outputs = vgg_model.layers[-1].output)
  image = load_img(filename, target_size=(224, 224))
  image = img_to_array(image)  # (224, 224, 3)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  feature = vgg_model.predict(image, verbose=0)
  label = decode_predictions(feature)[0][0][1].replace("_", " ")
  print(label)
  encode_prediction = embed([label])[0]
  euclidean_final_tag_prediction = ["#" + popular_tags[i].replace(" ", "") for i in tree.query(encode_prediction, k = 12)[1]]
  return euclidean_final_tag_prediction
 
# euclidean_predict_one_image("cat.jpg", tree, popular_tags)